In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,26502
2,Huelva,Confirmados PDIA 14 días,1239
3,Huelva,Tasa PDIA 14 días,"241,4404583276497"
4,Huelva,Confirmados PDIA 7 días,813
5,Huelva,Total Confirmados,26689
6,Huelva,Curados,21894
7,Huelva,Fallecidos,355


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  26502.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  6920.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 773 personas en los últimos 7 días 

Un positivo PCR cada 575 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,26502.0,813.0,1239.0,513170.0,158.427032,241.440458,370.0
Condado-Campiña,7922.0,413.0,696.0,156231.0,264.352145,445.494172,196.0
Huelva-Costa,15789.0,340.0,461.0,289548.0,117.424399,159.213671,165.0
Huelva (capital),6920.0,186.0,250.0,143837.0,129.313042,173.807852,93.0
Almonte,1164.0,123.0,204.0,24507.0,501.897417,832.415228,58.0
Cartaya,1105.0,89.0,112.0,20083.0,443.160882,557.685605,42.0
Rociana del Condado,417.0,36.0,71.0,7939.0,453.457614,894.319184,26.0
Moguer,955.0,63.0,86.0,21867.0,288.105364,393.286688,26.0
Sierra de Huelva-Andévalo Central,2371.0,41.0,57.0,67391.0,60.838984,84.581027,18.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
San Bartolomé de la Torre,266.0,29.0,49.0,3761.0,771.071524,1302.844988,14.0
Rociana del Condado,417.0,36.0,71.0,7939.0,453.457614,894.319184,26.0
Almonte,1164.0,123.0,204.0,24507.0,501.897417,832.415228,58.0
Sanlúcar de Guadiana,6.0,3.0,3.0,403.0,744.416873,744.416873,NaN
Bonares,218.0,24.0,37.0,6060.0,396.039604,610.561056,15.0
Cartaya,1105.0,89.0,112.0,20083.0,443.160882,557.685605,42.0
Condado-Campiña,7922.0,413.0,696.0,156231.0,264.352145,445.494172,196.0
Niebla,182.0,6.0,18.0,4116.0,145.772595,437.317784,4.0
Bollullos Par del Condado,706.0,20.0,58.0,14387.0,139.014388,403.141725,11.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Niebla,182.0,6.0,18.0,4116.0,145.772595,437.317784,4.0,0.333333
Bollullos Par del Condado,706.0,20.0,58.0,14387.0,139.014388,403.141725,11.0,0.344828
Paterna del Campo,123.0,3.0,8.0,3457.0,86.780445,231.414521,0.0,0.375000
Ayamonte,1251.0,4.0,10.0,21104.0,18.953753,47.384382,1.0,0.400000
Aroche,85.0,2.0,5.0,3054.0,65.487885,163.719712,1.0,0.400000
Gibraleón,555.0,9.0,21.0,12737.0,70.660281,164.873989,3.0,0.428571
Lepe,2035.0,10.0,21.0,27880.0,35.868006,75.322812,4.0,0.476190
Calañas,173.0,1.0,2.0,2768.0,36.127168,72.254335,1.0,0.500000
Higuera de la Sierra,31.0,1.0,2.0,1291.0,77.459334,154.918668,0.0,0.500000
